## Imports

In [1]:
# Modules
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.graph_objs as go
import statsmodels.tsa.api as smt
import statsmodels.tsa.stattools as stt
import os
import shutil
import sklearn
from sklearn import preprocessing
import logging
import itertools

# Tensorflow
import tensorflow as tf
# Disable tf log
tf.logging.set_verbosity(tf.logging.WARN)

# Pyplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Matplotlib
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Logger
logger = logging.getLogger('bitcoin')
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s', datefmt='%m-%d %H:%M')
handler.setFormatter(formatter)
logger.addHandler(handler)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



## Loading data

In [2]:
# Download data from various sources. Note that iloc is to reverse the df, 
# so that recent data are on bottom

# Crypto time series
btc = web.DataReader('BITFINEX/BTCUSD', 'quandl').iloc[::-1]
eth = web.DataReader('BITFINEX/ETHUSD', 'quandl').iloc[::-1]
ltc = web.DataReader('BITFINEX/LTCUSD', 'quandl').iloc[::-1]
bch = web.DataReader('BITFINEX/BCHUSD', 'quandl').iloc[::-1]
xrp = web.DataReader('BITFINEX/XRPUSD', 'quandl').iloc[::-1]

# Bitcoin generic infor dataset
info = web.DataReader('BITCOINWATCH/MINING', 'quandl').iloc[::-1]

In [3]:
btc.describe()

,High,Low,Mid,Last,Bid,Ask,Volume
count,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000
mean,2445.582191,2250.663332,2360.725997,2360.746854,2360.411044,2361.040951,28045.150961
std,3734.025299,3362.574915,3571.668353,3571.583703,3571.362447,3571.974408,27982.241498
min,0.000000,0.000000,177.980000,178.000000,177.960000,178.000000,0.000000
25%,376.895000,353.402500,366.013750,366.265000,365.747500,366.280000,9809.994150
50%,610.425000,591.245000,601.852500,603.820000,601.020000,602.790000,19460.478814
75%,2617.250000,2423.000000,2534.487500,2533.975000,2534.275000,2534.700000,37165.152130
max,19891.000000,18622.000000,19209.000000,19210.000000,19208.000000,19210.000000,269676.763207


In [4]:
info.describe()

,TotalBTC,MarketCap,Transactionslast24h,Transactionsavgperhour,Bitcoinssentlast24h,Bitcoinssentavgperhour,Count,Blockslast24h,Blocksavgperhour,Difficulty,NextDifficulty,NetworkHashrateTerahashs,NetworkHashratePetaFLOPS
count,1.635000e+03,1.635000e+03,1212.000000,1212.000000,1.212000e+03,1.212000e+03,1635.000000,1635.000000,1635.000000,1.635000e+03,1.635000e+03,1.635000e+03,1.635000e+03
mean,1.487077e+07,3.588314e+10,152519.490924,6354.979579,1.802472e+06,7.510299e+04,394660.728440,114.781040,4.783052,3.205499e+11,3.169317e+11,2.268687e+06,2.881232e+07
std,1.564912e+06,5.754576e+10,88570.085157,3690.420212,2.569819e+06,1.070758e+05,72879.249481,70.357279,2.931830,3.556939e+11,3.451909e+11,2.470975e+06,3.138138e+07
min,1.160878e+07,1.342254e+09,1407.000000,58.630000,1.395002e+04,5.812500e+02,254350.000000,0.000000,0.000000,6.575006e+07,6.849834e+07,4.903300e+02,6.227190e+03
25%,1.352329e+07,5.120081e+09,71802.500000,2991.767500,7.857671e+05,3.274029e+04,330930.500000,0.000000,0.000000,3.960367e+10,3.962895e+10,2.836750e+05,3.602673e+06
50%,1.542145e+07,8.622801e+09,115395.000000,4808.125000,1.310239e+06,5.459329e+04,406857.000000,146.000000,6.080000,1.668515e+11,1.746104e+11,1.249910e+06,1.587386e+07
75%,1.635462e+07,3.438373e+10,228695.750000,9528.992500,2.263041e+06,9.429339e+04,468369.000000,160.000000,6.670000,5.959219e+11,6.175699e+11,4.420738e+06,5.614337e+07
max,1.652280e+07,3.172873e+11,372103.000000,15504.290000,6.735430e+07,2.806429e+06,481823.000000,257.000000,10.710000,9.232331e+11,9.316186e+11,6.668786e+06,8.469358e+07


## LSTM

In [5]:
# Select features to be used in prediction
features = {'ltc_Mid': ltc.Mid, 'btc_Mid': btc.Mid, 'btc_Volume': btc.Volume,
            'info_MarketCap': info.MarketCap}

### Plots

In [6]:
data = []
d = {}
for ind, (key, value) in enumerate(features.items()):
    # Create trace to add to data
    trace = go.Scatter(x=value.index, y=value, name=key, yaxis='y{}'.format(ind+1))
    data.append(trace)
    # Create dictionaries to add layout
    d['yaxis{}'.format(ind+2)] = {'overlaying': 'y', 'side': 'left'}
fig = go.Figure(data=data, layout=d)
iplot(fig)

In [7]:
# Construct the dataframe basing on features to use during prediction
tms = pd.DataFrame(index = btc.index)
for key, values in features.items():
    tms[key] = values
logger.info('Shape of tms %s', tms.shape)
tms = tms.dropna()
logger.info('Shape of tms after filtering not NaNs rows %s', tms.shape)
tms_filtered = tms.copy()

07-21 18:03 bitcoin      INFO     Shape of tms (1526, 4)
07-21 18:03 bitcoin      INFO     Shape of tms after filtering not NaNs rows (1410, 4)


#### Process the input

In [8]:
def sentences_generator(sequence, seq_len = 40, step=1):
    sentences = []
    next_tokens = []
    for i in range(0, len(sequence) - seq_len, step):
        sentences.append(sequence[i: i + seq_len])
        next_tokens.append(sequence[i + seq_len])
    logger.info('Sequence length: %s' % len(sentences))
    logger.info('Next token length: %s' % len(sentences))
    return np.array(sentences), np.array(next_tokens)

def normalize(data, regressors, kind = 'min_max'):
    '''Normalize data with a standard scaler'''
    # Dataframes
    scalers = []
    for el in regressors:
        if kind == 'min_max':
            scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
        elif kind == 'standard':
            scaler = sklearn.preprocessing.StandardScaler()
        data[el] = scaler.fit_transform(data[el].values.reshape(-1,1))
        scalers.append(scaler)
    return data, scalers

def permutate_sequences(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

###### Normalizing data

In [9]:
tms, scaler = normalize(tms.copy(), tms.columns)

##### Creating sequences

In [10]:
# Number of previous timesteps to consider in predictions
timesteps = 10

In [11]:
# Create seguences which can be feeded to TF
sequences, next_tokens = sentences_generator(tms.values, seq_len = timesteps)
# Keep track of original dates
dates = tms_filtered.index[timesteps:]

07-21 18:03 bitcoin      INFO     Sequence length: 1400
07-21 18:03 bitcoin      INFO     Next token length: 1400


##### Splitting train, val, test

In [12]:
# Sequence splitting
test_split = int(len(sequences)*.90)
val_split = int(len(sequences)*.75)
train_sequences, train_next_tokens = sequences[:val_split], next_tokens[:val_split]
val_sequences, val_next_tokens = sequences[val_split:test_split], next_tokens[val_split:test_split]
test_sequences, test_next_tokens = sequences[test_split:], next_tokens[test_split:]
train_dates, val_dates, test_dates = dates[:val_split], dates[val_split:test_split], dates[test_split:]

#### Init params

In [13]:
# Reset the graph
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
num_epoch = 1000
batch_size = 128
display_meas_step = 20
save_model_step = 50

# Network Parameters
# Number of features
num_input = len(features)
# Number of features yielded from LSTM hidden layer
num_hidden_rnn = 256
# Number of features yielded from first ANN layer
num_hidden_nn = 32
# Number of output features
num_output = len(features)

# TF inputs
X = tf.placeholder("float", [None, timesteps, num_input], name = 'X')
Y = tf.placeholder("float", [None, num_output], name = 'Y')

#### Model

In [14]:
# Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
with tf.name_scope('input'):
    x = tf.unstack(X, timesteps, axis = 1)
with tf.name_scope('rnn'):
    # Define a lstm cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden_rnn, forget_bias=1.0)
    # Get LSTM cell outputs and states
    rnn_outputs, rnn_states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    # For tensorboard
    rnn_weights = lstm_cell.weights[0]
    rnn_biases = lstm_cell.weights[1]
    tf.summary.histogram("outputs", rnn_outputs)
    tf.summary.histogram("states", rnn_states)
    tf.summary.histogram("rnn_weights", rnn_weights)
    tf.summary.histogram("rnn_biases", rnn_biases)
with tf.name_scope('nn'):
    # Define the first ANN
    nn_outputs = tf.layers.dense(rnn_outputs[-1], num_hidden_nn, activation=tf.nn.tanh, name = 'nn1')
    # Define the second ANN
    prediction = tf.layers.dense(nn_outputs, num_output, name = 'nn2')
    # For tensorboard
    with tf.variable_scope("nn1", reuse=True):
        nn1_weights = tf.get_variable('kernel')
        nn1_biases = tf.get_variable('bias')
    with tf.variable_scope("nn2", reuse=True):
        nn2_weights = tf.get_variable('kernel')
        nn2_biases = tf.get_variable('bias')
    tf.summary.histogram("nn1_weights", nn1_weights)
    tf.summary.histogram("nn1_biases", nn1_biases)
    tf.summary.histogram("nn2_weights", nn2_weights)
    tf.summary.histogram("nn2_biases", nn2_biases)
    tf.summary.histogram("train_prediction", prediction)

In [15]:
# Loss
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(prediction - Y), name = 'loss')
    train_loss_summ = tf.summary.scalar('train_loss', loss, collections = 'evaluation')
    val_loss_summ = tf.summary.scalar('val_loss', loss, collections = 'evaluation')
# Optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate, name = 'optimizer')
# Minimize
with tf.name_scope('train'):
    minimize = optimizer.minimize(loss, name = 'minimize')

#### Training

In [ ]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Select a directory where saving the results
base_dir = './rnn'
file_writer_dir = '{}/run1/logs'.format(base_dir)
save_model_dir = '{}/run1/model'.format(base_dir)
# Delete the directory if it exists
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

In [ ]:
try:
    # Start training
    with tf.Session() as sess:
        # Run the initializer
        sess.run(init)
        # Merge all the summaries (except those having no default 'collections')
        merged_summary = tf.summary.merge_all()
        # Write the summary handler for tensorboard
        writer = tf.summary.FileWriter(file_writer_dir)
        writer.add_graph(sess.graph)
        
        # Training along the epochs
        for epoch in range(1, num_epoch+1):
            for start in range(0, len(train_sequences) + 1, batch_size):
                logger.debug('Taking batch from {} to {}'.format(start, min(start+batch_size, len(train_sequences))))
                # Creating training batches
                batch_x = train_sequences[start:min(start+batch_size, len(train_sequences))]
                batch_y = train_next_tokens[start:min(start+batch_size, len(train_next_tokens))]
                
                # Shuffling the data at each epoch 
                batch_x, batch_y = permutate_sequences(batch_x, batch_y)
                
                # Run optimization operation (backprop)
                sess.run(minimize, feed_dict={X: batch_x, Y: batch_y})
            # Execute all the summaries
            summ = sess.run(merged_summary, feed_dict={X: batch_x, Y: batch_y})
            writer.add_summary(summ, epoch)
            if epoch % display_meas_step == 0 or epoch == 1:
                # Calculate batch loss and accuracy
                train_loss, train_prediction, train_summ = sess.run([loss, prediction, train_loss_summ], 
                                                           feed_dict={X: train_sequences, Y: train_next_tokens})
                val_loss, val_prediction, val_summ = sess.run([loss, prediction, val_loss_summ], 
                                                       feed_dict={X: val_sequences, Y: val_next_tokens})
                # Execute the summaries having no default 'collections'
                writer.add_summary(train_summ, epoch)
                writer.add_summary(val_summ, epoch)
                logger.info("Epoch {}, minibatch training loss={:.6f}, val loss={:.6f}".
                             format(str(epoch), train_loss, val_loss))
            if epoch % save_model_step == 0 or epoch == 1:
                # Save the model
                logger.info('Saving the model')
                tf.saved_model.simple_save(sess,
                                           '{}/epoch{}'.format(save_model_dir, epoch),
                                           inputs={"x": X},
                                           outputs={"y": Y})
        print("Optimization Finished!")
except KeyboardInterrupt as e:
    logger.warning('Execution interrupted')

07-21 18:04 bitcoin      INFO     Epoch 1, minibatch training loss=0.094903, val loss=0.107988
07-21 18:04 bitcoin      INFO     Saving the model
07-21 18:04 bitcoin      INFO     Epoch 20, minibatch training loss=0.008930, val loss=0.040286
07-21 18:04 bitcoin      INFO     Epoch 40, minibatch training loss=0.008185, val loss=0.036630
07-21 18:04 bitcoin      INFO     Saving the model
07-21 18:04 bitcoin      INFO     Epoch 60, minibatch training loss=0.010638, val loss=0.033474
07-21 18:04 bitcoin      INFO     Epoch 80, minibatch training loss=0.007799, val loss=0.034190
07-21 18:04 bitcoin      INFO     Epoch 100, minibatch training loss=0.007610, val loss=0.031787
07-21 18:04 bitcoin      INFO     Saving the model
07-21 18:04 bitcoin      INFO     Epoch 120, minibatch training loss=0.008591, val loss=0.034247
07-21 18:04 bitcoin      INFO     Epoch 140, minibatch training loss=0.007856, val loss=0.033588
07-21 18:04 bitcoin      INFO     Saving the model
07-21 18:05 bitcoin      I

#### Predicting

In [ ]:
# Specify the model to use for predictions
checkpoint_epoch = 150
graph = tf.Graph()
# Define the number of days ahead for which performing predictions
ahead_steps = 30
# Define those starting points from which beginning ahead predictions
startings = [x for x in range(0, len(sequences), 40)] + [len(sequences)-1]
with graph.as_default():
    with tf.Session(graph=graph) as sess:
        # Restore saved model
        logger.info('Restoring...')
        tf.saved_model.loader.load(sess, 
                                   [tf.saved_model.tag_constants.SERVING], 
                                   '{}/epoch{}'.format(save_model_dir, checkpoint_epoch))
        logger.info('Restored...')
        # Restore saved values
        X = graph.get_tensor_by_name("X:0")
        Y = graph.get_tensor_by_name("Y:0")
        prediction = graph.get_tensor_by_name('nn/nn2/BiasAdd:0')
        loss = graph.get_tensor_by_name('loss/loss:0')
        # Predict for training, validation and test
        train_loss, train_prediction = sess.run([loss, prediction], feed_dict={X: train_sequences, 
                                                                         Y: train_next_tokens})
        val_loss, val_prediction = sess.run([loss, prediction], feed_dict={X: val_sequences, 
                                                                     Y: val_next_tokens})
        test_loss, test_prediction = sess.run([loss, prediction], feed_dict={X: test_sequences, 
                                                                       Y: test_next_tokens})

        # Predict next element and progressively shifting predictions
        all_ahead_prediction, all_ahead_dates = [], []
        for starting in startings:
            ahead_sequences = np.expand_dims(sequences[starting], axis = 0)
            ahead_next_tokens = np.expand_dims(next_tokens[starting], axis = 0)
            ahead_prediction = []
            for ah_step in range(ahead_steps):
                pred = sess.run([prediction], feed_dict = {X: ahead_sequences})
                ahead_sequences = np.concatenate((ahead_sequences[:, 1:, :], pred), axis = 1)
                ahead_prediction.append(pred[0][0])
            ahead_prediction = np.array(ahead_prediction)
            all_ahead_prediction.append(ahead_prediction)

In [ ]:
# Create dataframes containing predictions for 
# training, validation and test
train_preds = pd.DataFrame({'set': 'training'}, index = tms_filtered.loc[train_dates].index)
val_preds = pd.DataFrame({'set': 'validation'}, index = tms_filtered.loc[val_dates].index)
test_preds = pd.DataFrame({'set': 'test'}, index = tms_filtered.loc[test_dates].index)
for i, el in enumerate(tms_filtered.columns):
    rebuilt_train = scaler[i].inverse_transform(train_prediction[:, i].reshape(-1, 1)).squeeze()
    train_preds['transf_preds_%s' % el] = train_prediction[:, i]
    train_preds['preds_%s' % el] = rebuilt_train
    rebuilt_val = scaler[i].inverse_transform(val_prediction[:, i].reshape(-1, 1)).squeeze()
    val_preds['transf_preds_%s' % el] = val_prediction[:, i]
    val_preds['preds_%s' % el] = rebuilt_val
    rebuilt_test = scaler[i].inverse_transform(test_prediction[:, i].reshape(-1, 1)).squeeze()
    test_preds['transf_preds_%s' % el] = test_prediction[:, i]
    test_preds['preds_%s' % el] = rebuilt_test

# Create dataframe containing ahead predictions
all_ahead_preds = []
for starting, ahead_prediction in zip(startings, all_ahead_prediction):
    ahead_preds = pd.DataFrame(index = pd.date_range(dates[starting],
                                                     periods=ahead_steps, freq='D'))
    for i, el in enumerate(tms.columns):
        rebuilt_ahead = scaler[i].inverse_transform(ahead_prediction[:, i].reshape(-1, 1)).squeeze()
        ahead_preds['transf_preds_%s' % el] = ahead_prediction[:, i]
        ahead_preds['preds_%s' % el] = rebuilt_ahead

    all_ahead_preds.append(ahead_preds)
     
# Sort dataframe
train_preds = train_preds.sort_index()
val_preds = val_preds.sort_index()
test_preds = test_preds.sort_index()

#### Plot predictions

In [ ]:
for el in tms_filtered.columns:
    # Trace for training
    training = go.Scatter(
        x = train_preds.index,
        y = train_preds['preds_%s' % el],
        name = 'Training %s' % el
    )
    # Trace for validation
    validation = go.Scatter(
        x = val_preds.index,
        y = val_preds['preds_%s' % el],
        name = 'Validation %s' % el
    )
    # Trace for test
    test = go.Scatter(
        x = test_preds.index,
        y = test_preds['preds_%s' % el],
        name = 'Test %s' % el
    )
    # Trace for ground truth
    ground_truth = go.Scatter(
        x = tms_filtered.index,
        y = tms_filtered[el],
        name = 'Ground truth %s' % el,  
    )
    # Trace for ahead predictions
    aheads = []
    for ahead_preds in all_ahead_preds:
        ahead = go.Scatter(
            x = ahead_preds.index,
            y = ahead_preds['preds_%s' % el],
            showlegend=False,
            marker = dict(color = 'rgba(80, 26, 80, 0.8)')
        )
        aheads.append(ahead)
    data = [training, validation, test, ground_truth] + aheads
    layout = go.Layout(
        title = 'Predictions %s ' % el,
        yaxis = dict(title = 'Last')
    )
    fig = go.Figure(data = data, layout = layout)
    iplot(fig)

In [ ]:
# Aggregate the dataframes
preds = pd.concat([train_preds, val_preds, test_preds])
for el in tms.columns:
    preds['true_%s' % el] = tms[el]
    preds['error_%s' % el] = preds['preds_%s' % el] - preds['true_%s' % el]
preds['error'] = abs(preds[[x for x in preds.columns if 'error' in x]]).sum(axis = 1)

fig, ax = plt.subplots(ncols = 2, figsize = (18, 6))
# Filter out extreme error percentiles and plot
small_perc, large_perc = preds['error'].quantile([0.10, 0.90]).values
no_outliers = preds.loc[(preds.error > small_perc) & (preds.error < large_perc), 'error']
sns.distplot(no_outliers, ax = ax[0])
sns.boxplot(no_outliers, ax = ax[1])
display(preds['error'].quantile([0, 0.5, 0.75, 0.9, 0.95, 1]))